# Homework 1- Group 5

### Descripition on model and choices

<b> MongoDB: </b> <br> 
We use the mongoDB to store 3 types of data in 1 collection. The 3 types are: <br>
- companies: save details on company- Company Name, Company Description, Jobs List
- jobs: save details on job- Job Id, Job Title, Location, Requirements List, Publish Date, Status, Applications List
- applications: save details on application- Candidate Name, Email, LinkedIn URL, Skills List, Application Date

For use 1 collection we save for each company a list of all the jobs it has and for each job a list of all the applications it has. <br><br>

<b> Redis: </b> <br> 
We use the RedisDB to store attributes that we could read in fast way. In the mongoDB we don't know the meaning of the values so it more difficult to scan and search a string/number in the value. Therefore, all the data that we could get from calling his key that was difficilut to search in the MongoDB we save in the redis.
<br> All the attributes we save in the redis are:
- for each comapny name we save the last job id we used.
- for each company name we save the jobs id where the status is open
- for each location's and title's job we save the job id and company name that answer this details and with open status
- for each company name, specific job and specific email's candidate we save the number of same skills (skill and requierment)

<br> as we said before, all this attributes are complex and redis allow as to get the value faster then mongoDB,

-----------------------------------

<b> Assumptions: </b>
1. We can add a job with the same details as an existing job to a company. They will be with different id's.
2. report 1 - We return only the relevant lines, only existing dates from 2020 that are in the collection.
3. report 1 - The date we present is the publish date of the job (not last modified date).
------------------

In [1]:
# import packages we need
import pymongo
from pymongo import MongoClient
import redis
import pandas as pd
from IPython.core import display
import datetime

In [2]:
# create MongoDB client
client = MongoClient()

In [3]:
# create a new DB in Mongo
db = client['stud27']
# create a collection - the name is website
collection = db.website

In [4]:
# create redis DB
r = redis.StrictRedis(host='bdl1.eng.tau.ac.il', port=6379)

### operation 1

In [5]:
# the function add new company with unique name.
def add_company(company):
    # get the name of the company
    name = company["company_name"]
    # check if the company name is unique (doesn't exist in the DB)
    if collection.find_one({"company_name": name}) is None:
        # initialize an empty list for the jobs that we will add to this company
        company["jobs_list"] = []  
        # insert the company to the MongoDB
        collection.insert_one(company)
        # initialize in redis last job id thwt we used to 0
        r.set(name+':last_job_id',0)
        print 'intiliaze the last job id in redis for company %s' % name 
        print 'The company %s added to the collection with the following details' % name
        display_comp = pd.DataFrame([company])
        return  display.display(display_comp)
    # inform if the name is not unique 
    else: 
        return "Please choose another name. This name %s is already exist in this collection" % name

### operation 2

In [6]:
# this function get job and company name and add the job to the job list in the company and save it with unique id
def add_job(job,company_name):
    # check if the company exist
    current_company = collection.find_one({'company_name':company_name})
    # if exist:
    if current_company != None:
        # get the last id we used for job in this company from redis and increase it by 1 (will be unique)
        job_id = int(r.incr(company_name+':last_job_id'))
        # set job_id for the new job
        job['job_id'] = str(job_id)
        # initialize an empty list for the applications that we will add for this job in this company
        job['applications_list'] = []
        # save unique job id with the company name for reports
        job['_id'] = 'job_%s_%s' %(str(job_id),company_name)
        job['status'] = job['status'].lower()
        # find the company with this company name and add the job to its job list
        collection.update_one({ "company_name": company_name},
                        {"$push": {'jobs_list': job}})
        
        # check if we have date for this job
        if 'publish_date' in job:
            # covert the date from srting to datetime object (for reports)
             job['publish_date'] = datetime.datetime.strptime(job['publish_date'], "%m-%d-%Y")
        
        # if the status for this job is open
        if job['status'].lower()=='open':
            # save for each company name the jobs id with open status in redis
            r.sadd('open_jobs:company_name_'+company_name,job_id)
            # save for each location and title the company name and job id in redis only for open jobs
            r.sadd('location,title:'+job['location']+','+job['job_title'],str(job_id)+'!'+company_name)
            print 'The open jobs updated in redis- added job id(by comapny & by location and title)'
            
        print "The job with id %d in company %s added to the collection with the foloowing details" % (job_id, company_name)
        display_job = pd.DataFrame([job])
        return display.display((display_job))
    
    # if the company is not exist        
    else:
        return "This company name- %s is not exist in this collection" % company_name

### operation 3

In [7]:
# this function add new application to a job with open status and only if the application isn't register yet
# to this job id in the company name. in addition each email will be unique.
def add_application(candidate, job_id, company_name):
    # find  all the jobs for a company name if exist
    companies = collection.find_one({"company_name": company_name})
    if companies != None:
        current_jobs = companies['jobs_list']
        # check if the job_id in all the jobs we got
        current_job = [job for job in current_jobs if job['job_id'] == job_id ]
        # if there is not this job_id
        if len(current_job) == 0: 
            return "This job id %s is not exist in this company %s, please try again" % (job_id, company_name)
        # if there is this job id
        current_job = current_job [0]
        # if the status is close
        if current_job['status'] == 'close':
            return "Can not register to the job %s in company %s because it is close for application" % (job_id, company_name)
        # check if the email is unique and doesn't exist
        if candidate['email'] in [i['email'] for i in current_job['applications_list']]:
            return "The candidate's email (%s) exist for this job, please try again with another candidate" % candidate['email']
        # if can register an application for this job
        else: 
            # check if we have date for this candidate
            if 'application_date' in candidate:
                # covert the date from srting to datetime object (for reports)
                 candidate['application_date'] = datetime.datetime.strptime(candidate['application_date']+'.000Z', "%m-%d-%Y  %H:%M:%S.000Z")
            # find the company with this company name and job id and add the candidate to its application list
            collection.update_one({ "company_name": company_name, "jobs_list.job_id": job_id},
                            {"$push": {'jobs_list.$.applications_list': candidate}})

            # save the requierments for the job
            job_requirements=current_job['requirements']
            # save the number of shared skills of the candidate and requierments of the job
            intersection=len(set(candidate['skills']).intersection(job_requirements))
            # save for each company name. job id and candidate the number of shared skills
            r.zadd('intersection:company_name_'+company_name+',job_id:'+str(job_id),intersection, candidate['email'])
            print 'The number of matched skills updated in redis by comapny, job id and candidate'
            
            print "The candidate with the following email: %s added to the collection for the job id %s in company %s" % (candidate['email'],job_id, company_name)
            display_candidate = pd.DataFrame([candidate])
            return  display.display(display_candidate)
    # if compant is not exist
    else:
        return 'The company %s is not exist on this collection' % company_name


### operation 4

In [8]:
# this function get company name. job id and new status and change the current status to the new one.
def update_job_status(company_name, job_id, new_status):
    # check if the new_status is in the correct options    
    if new_status.lower() not in ('open','close'):
        return 'The new status is Invalid'
    # if the new status is ok
    else:
        # check if the company is exist
        current_company = collection.find_one({'company_name':company_name})
        # if not exisit
        if current_company is None:
            return 'This company %s is not exist' % company_name
        # if exist
        else:
            # check if the job id is not exist in this company
            if job_id not in [i['job_id'] for i in current_company['jobs_list']]:
                return 'This job %s is not exist in this company %s' % (job_id, company_name)
            # if the job id is exist in this company
            else:
                # check if the job id exist in the open jobs for this company
                status_open = r.sismember('open_jobs:company_name_'+company_name,job_id)
                # if job with open status and the new status is open too
                if status_open and new_status.lower() == 'open':
                    return 'Nothing to update- the status is already open'
                # if job with close status and the new status is close too
                elif not status_open and new_status.lower() == 'close':
                    return 'Nothing to update- the status is already close'
                # find the company with this company name and job id and update the status
                # add last midified date for this change
                collection.update_one({ "company_name": company_name, "jobs_list.job_id": job_id},
                                {"$set": {"jobs_list.$.status": new_status.lower()},
                                 "$currentDate":{"jobs_list.$.lastModified":True}})
                
                # get the company where the company name is this and it has the job id in job_list
                company = collection.find_one({"company_name": company_name, "jobs_list.job_id": job_id})
                # save the job list and get the title and the location for this job id
                job_list = company['jobs_list']
                title = [job['job_title'] for job in job_list if job['job_id']==job_id][0]
                location = [job['location'] for job in job_list if job['job_id']==job_id][0]
                
                # if the new status is open
                if new_status.lower() =='open':
                    # add the job id to the open jobs for this company in redis
                    r.sadd('open_jobs:company_name_'+company_name,job_id)
                    # add for this location and title the comapny name with the job id
                    r.sadd('location,title:'+str(location)+','+str(title),str(job_id)+'!'+company_name)
                    print 'The open jobs updated in redis- add job id(by comapny & by location and title)'
                else:
                    # remove the job id to the open jobs for this company in redis
                    r.srem('open_jobs:company_name_'+company_name,job_id)
                    # remove for this location and title the comapny name with the job id
                    r.srem('location,title:'+str(location)+','+str(title),str(job_id)+'!'+company_name)
                    print 'The open jobs updated in redis- remove job id(by comapny & by location and title)'
                
                return "the job id %s of company %s changed the status to %s" % (job_id,company_name,new_status)
                    

### operation 5

In [9]:
# this function get location and title and return the number of open jobs using redis
def show_number_of_jobs(location,job_title):
    # get the correct number from redis
    num_open_jobs = len(r.smembers('location,title:'+str(location)+','+str(job_title)))
    return 'The number of open jobs in location %s with title %s is %d' % (location, job_title, num_open_jobs)

### operation 6

In [10]:
# this function get company name and job id and return candidates's email sorts in asce order
# by number of matches between skills and requierments
def show_candidates(company_name, job_id):
    # check if the company name with this job id exist
    current_company = collection.find_one({"company_name": company_name, 'jobs_list.job_id': job_id})
    # if exist
    if current_company is not None:
        # get the candidates's emails for this job in this company from redis
        candidate = r.zrange('intersection:company_name_'+company_name+',job_id:'+str(job_id),0,-1)
        if len(candidate) == 0:
            return ('There is no candidates in this job %s in the company %s' % (job_id, company_name))
        print ('These are the emails in ASCE order by number of matches')
        return  display.display(pd.DataFrame(candidate, columns=['emails']))
    # if company and job id is not exist
    else: 
        return ('There is no a company %s with this job id %s - This job is not exist for this company or company is not exist' % (company_name, job_id))

### report 1

In [11]:
# this function return for each date in 2020 that exist in the collection in the publish date of jobs
# the number of close jobs and the number of open jobs
def count_jobs_by_date(): 
    # split the collection by job_list
    unwind_func = {'$unwind': '$jobs_list'}
    # leave only _id of job , status and puclish date and add new 2 fields for open jobs and close and save only the year of publish date
    proj_func={'$project': {'jobs_list._id': 1, 'jobs_list.status':1, 'jobs_list.publish_date':1,
                            'year':{'$substr': [ '$jobs_list.publish_date', 6,-1 ]},
                            'num_open_jobs': {'$cond': [{'$eq': ['$jobs_list.status', 'open']}, 1, 0]},
                            'num_close_jobs': {'$cond': [{'$eq': ['$jobs_list.status', 'close']}, 1, 0]}}} 
    # leave only rows with 2020 in the year of the publish date
    match = {'$match': {'year':{'$eq':'2020'}}}
    # group by publish date and sum the numner of open jobs and close jobs sepreatly
    group_func ={'$group': {'_id': '$jobs_list.publish_date','num_open_jobs': {'$sum': '$num_open_jobs'},
                            'num_close_jobs': {'$sum': '$num_close_jobs'} } }
    # the _id now is the date so sort in ASCE order
    sort_func= { '$sort' : { '_id' : 1 }}
    # define the pipeline with all the functions
    pipeline = [unwind_func, proj_func,match, group_func]
    # execute the query and convert to data frame
    res_df = pd.DataFrame.from_dict(list(collection.aggregate(pipeline)))

    return display.display(res_df)

### report 2

In [12]:
# this function return for each job the number of application it has with application date in the past 30 days
def count_candidates_by_job():
    # create agg functions for aggregate() method with the pipeline
    # split the collection by job_list- job _id
    unwind_func = {'$unwind': '$jobs_list'}
    unwind2_func = {'$unwind': '$jobs_list._id'}
    # leave only _id of job and application list only with the application with the correct date
    proj_func={'$project': {'jobs_list._id': 1, 'jobs_list.applications_list': {'$filter': {'input': "$jobs_list.applications_list",'as': "list",
                                                                   'cond': { '$gt': [ "$$list.application_date", datetime.datetime.today()-datetime.timedelta(days=30) ] }}}}}
    # caculate the size of each application list
    group_func = {"$group": {"_id": "$jobs_list._id",'number_of_application':{'$first':{'$size': '$jobs_list.applications_list' }}}}
    
    # define the pipeline with all the functions
    pipeline = [unwind_func, unwind2_func,proj_func, group_func]
    # execute the query and convert to data frame
    res_df = pd.DataFrame.from_dict(list(collection.aggregate(pipeline)))
    print "\nThe number of applications who their application date is in the past 30 days for each job's id"
    return  display.display(res_df)

### recovery for redis

In [13]:
# this function will be run whenever the redis dever restarts (like power failure)
# the function reload all the data to cahce (redis) from the persistent data (mongodb)
def recovery():
    # recover the last job_id for each company
    # split by job_list and group by company name and save the maximum job_id
    unwind_func ={"$unwind": "$jobs_list"}
    group_func = {"$group": {"_id": "$company_name", "last_id": {"$max": "$jobs_list.job_id"}}}
    # create the pipeline and apply it on the collection
    pipeline=[unwind_func, group_func]
    companies_last_id = collection.aggregate(pipeline)
    # for each company
    for company in companies_last_id:
        # save the last id in redis with the correct company name
        r.set(str(company['_id'])+':last_job_id',int(company['last_id']))
        
    # recover all the things that related to open jobs
    # leave only job id , status, company name, title and location
    proj_func = {'$project': { 'jobs_list.status':1, 'jobs_list.job_id':1, 'company_name':1,
                              'jobs_list.job_title':1,'jobs_list.location':1}} 
    # leave only rows with open status
    match_func = {'$match': {'jobs_list.status':{'$eq':'open'}}}
    # create another pipeline and apply it on the collection
    pipeline2 = [unwind_func, proj_func, match_func]
    companies_open_jobs = list(collection.aggregate(pipeline2))
    # for each open job
    for open_job in companies_open_jobs:
        # save job_id, company name, title and location
        job_id = open_job['jobs_list']['job_id']
        company_name = open_job['company_name']
        location = open_job['jobs_list']['location']
        title = open_job['jobs_list']['job_title']
        # recover the jobs id for open job for each company-add it to redis to a set
        r.sadd('open_jobs:company_name_'+str(company_name),int(job_id))
        # recover the jobs id and company name for open job for each title&location-add it to redis to a set
        r.sadd('location,title:'+str(location)+','+str(title),str(job_id)+'!'+str(company_name))
    
    # recover all the things that related to application so save the applications list for each job
    # leave only job_id , company_name, skills and requirements
    proj_func2 = {'$project': { 'jobs_list.job_id':1, 'company_name':1, 'jobs_list.applications_list.skills':1,
                       'jobs_list.applications_list.email':1, 'jobs_list.requirements':1}} 
    # split by application list
    unwind_func2 = {'$unwind': '$jobs_list.applications_list'}
    # leave only job_id , company_name and number of matches between skills and requirements
    proj_func3 = {'$project': { 'jobs_list.job_id':1, 'company_name':1,'jobs_list.applications_list.email':1,
                        'num_matches':{'$size':{'$setIntersection':['$jobs_list.requirements','$jobs_list.applications_list.skills']}}}}
    # create another pipeline and apply it on the collection
    pipeline3 = [unwind_func, proj_func2, unwind_func2, proj_func3] 
    applications = list(collection.aggregate(pipeline3))
    # for each application
    for application in applications:
        # save job_id, company_name, number of matches and email
        job_id = application['jobs_list']['job_id']
        company_name = application['company_name']
        num_of_matches = application['num_matches']
        email = application['jobs_list']['applications_list']['email']
        # recover the number of matches between requirements and skills for each application by company name and job id
        r.zadd('intersection:company_name_'+str(company_name)+',job_id:'+str(job_id),num_of_matches, str(email))
    
    return 'All the data recovered to redis succefully'

### execute

In [14]:
# this function call all the function we created above with all the examples from the instruction
def execute():
    print ('-------------------------------- operations --------------------------------')
    print ('\noperation 1')
    add_company({'company_name':'TAU', 'company_description':'University'})
    
    print ('\noperation 2')
    add_job({'job_title':'bi developer', 'location': 'Tel Aviv',
            'requirements':['python','big data','mongodb'],
            'status':'open','publish_date':'01-02-2020'},'TAU')

    print ('\noperation 3')
    add_application({'candidate_name':'laura', 'email':'laura@gmail.com',
                    'linkedin':'https://www.linkedin.com/in/laura/', 'skills': ['python','sql'],
                    'application_date':'01-02-2020 15:00:00'}, '1','TAU')
    
    print ('\noperation 4')
    display.display(update_job_status('TAU','1','close'))
    display.display(update_job_status('TAU','1','open'))
    
    print ('\noperation 5')
    display.display(show_number_of_jobs('Tel Aviv','bi developer'))
    
    print ('\noperation 6')
    show_candidates('TAU','1')
   
    print ('\n\n-------------------------------- reports --------------------------------')
    print ('\nreport 1')
    count_jobs_by_date()
    
    print ('\nreport 2')
    count_candidates_by_job()
    
    print ('\n\n-------------------------------- recovery --------------------------------')
    display.display(recovery())

--------------------------


In [15]:
# make sure all the DB-Mongo and Redis are empty and clean from old data
collection.drop()
r.flushall()

True

In [16]:
execute()

-------------------------------- operations --------------------------------

operation 1
intiliaze the last job id in redis for company TAU
The company TAU added to the collection with the following details


,_id,company_description,company_name,jobs_list
0,609e54768ec143509036df61,University,TAU,[]



operation 2
The open jobs updated in redis- added job id(by comapny & by location and title)
The job with id 1 in company TAU added to the collection with the foloowing details


,_id,applications_list,job_id,job_title,location,publish_date,requirements,status
0,job_1_TAU,[],1,bi developer,Tel Aviv,2020-01-02,"[python, big data, mongodb]",open



operation 3
The number of matched skills updated in redis by comapny, job id and candidate
The candidate with the following email: laura@gmail.com added to the collection for the job id 1 in company TAU


...

,_id,num_close_jobs,num_open_jobs
0,01-02-2020,0,1



report 2

The number of applications who their application date is in the past 30 days for each job's id


,_id,number_of_application
0,job_1_TAU,0




-------------------------------- recovery --------------------------------


'All the data recovered to redis succefully'